# Advanced Assignment: Data Analysis and Reporting Tool

This notebook contains a comprehensive data analysis and reporting tool that can handle both CSV and Excel files, perform various data manipulations, and generate insightful reports.

In [1]:
import pandas as pd
import numpy as np

np.random.seed(0)
dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')
products = ['Product A', 'Product B', 'Product C', 'Product D']
regions = ['North', 'South', 'East', 'West']

data = {
    'Date': np.random.choice(dates, 1000),
    'Product': np.random.choice(products, 1000),
    'Region': np.random.choice(regions, 1000),
    'Sales': np.random.randint(100, 1000, 1000),
    'Quantity': np.random.randint(1, 50, 1000)
}

df = pd.DataFrame(data)
df.to_csv('sample_sales_data.csv', index=False)
print("sample_sales_data.csv created successfully.")


sample_sales_data.csv created successfully.


In [1]:
!pip install openpyxl

  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 308.6 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

np.random.seed(0)
departments = ['Sales', 'Marketing', 'IT', 'HR', 'Finance']
job_levels = ['Junior', 'Mid-level', 'Senior', 'Manager']

data = {
    'Employee ID': range(1, 101),
    'Name': [f'Employee {i}' for i in range(1, 101)],
    'Department': np.random.choice(departments, 100),
    'Job Level': np.random.choice(job_levels, 100),
    'Salary': np.random.randint(30000, 120000, 100),
    'Years of Experience': np.random.randint(0, 20, 100)
}

df = pd.DataFrame(data)
df.to_excel('sample_employee_data.xlsx', index=False)
print("sample_employee_data.xlsx created successfully.")


sample_employee_data.xlsx created successfully.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.chart import BarChart, Reference, LineChart
import os
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [4]:
def read_file(file_path):
    """
    Read CSV or Excel file and return a pandas DataFrame.
    """
    try:
        if file_path.endswith('.csv'):
            return pd.read_csv(file_path)
        elif file_path.endswith(('.xlsx', '.xls')):
            return pd.read_excel(file_path)
        else:
            raise ValueError("Unsupported file format. Please use CSV or Excel files.")
    except Exception as e:
        logger.error(f"Error reading file: {e}")
        return None

def clean_data(df):
    """
    Clean the data by handling missing values and correcting data types.
    """
    try:
        # Remove rows with all NaN values
        df = df.dropna(how='all')
        
        # Fill NaN values with appropriate methods
        numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
        df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
        
        categorical_columns = df.select_dtypes(include=['object']).columns
        df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
        
        return df
    except Exception as e:
        logger.error(f"Error cleaning data: {e}")
        return None

def analyze_data(df):
    """
    Perform basic analysis on the data.
    """
    try:
        numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
        analysis = df[numeric_columns].agg(['mean', 'median', 'min', 'max'])
        return analysis
    except Exception as e:
        logger.error(f"Error analyzing data: {e}")
        return None

def group_data(df, group_by, agg_column):
    """
    Group data by a specified column and aggregate another column.
    """
    try:
        grouped = df.groupby(group_by)[agg_column].sum().sort_values(ascending=False)
        return grouped
    except Exception as e:
        logger.error(f"Error grouping data: {e}")
        return None

def create_bar_chart(data, title, xlabel, ylabel):
    """
    Create a bar chart from the given data.
    """
    plt.figure(figsize=(10, 6))
    data.plot(kind='bar')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.tight_layout()
    return plt

def create_line_chart(data, title, xlabel, ylabel):
    """
    Create a line chart from the given data.
    """
    plt.figure(figsize=(10, 6))
    data.plot(kind='line')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.tight_layout()
    return plt

def generate_excel_report(df, analysis, grouped_data, output_file):
    """
    Generate a comprehensive Excel report with multiple sheets.
    """
    wb = Workbook()
    
    # Summary sheet
    ws_summary = wb.active
    ws_summary.title = "Summary"
    ws_summary.append(["Data Analysis Report"])
    ws_summary.append(["Number of records", len(df)])
    ws_summary.append(["Number of columns", len(df.columns)])
    ws_summary.append(["Columns", ", ".join(df.columns)])
    
    # Style the summary sheet
    for row in ws_summary["A1:B4"]:
        for cell in row:
            cell.font = Font(bold=True)
    
    # Analysis sheet
    ws_analysis = wb.create_sheet(title="Analysis")
    for r in dataframe_to_rows(analysis, index=True, header=True):
        ws_analysis.append(r)
    
    # Style the analysis sheet
    for row in ws_analysis[f"A1:{chr(65+len(analysis.columns))}1"]:
        for cell in row:
            cell.font = Font(bold=True)
            cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
    
    # Grouped data sheet
    ws_grouped = wb.create_sheet(title="Grouped Data")
    for r in dataframe_to_rows(grouped_data.reset_index(), index=False, header=True):
        ws_grouped.append(r)
    
    # Create bar chart
    chart = BarChart()
    data = Reference(ws_grouped, min_col=2, min_row=1, max_row=len(grouped_data)+1)
    cats = Reference(ws_grouped, min_col=1, min_row=2, max_row=len(grouped_data)+1)
    chart.add_data(data, titles_from_data=True)
    chart.set_categories(cats)
    chart.title = "Grouped Data Chart"
    ws_grouped.add_chart(chart, "E2")
    
    # Raw data sheet
    ws_raw = wb.create_sheet(title="Raw Data")
    for r in dataframe_to_rows(df, index=False, header=True):
        ws_raw.append(r)
    
    # Style the raw data sheet
    for row in ws_raw[f"A1:{chr(65+len(df.columns)-1)}1"]:
        for cell in row:
            cell.font = Font(bold=True)
            cell.fill = PatternFill(start_color="DDDDDD", end_color="DDDDDD", fill_type="solid")
    
    # Save the workbook
    wb.save(output_file)
    logger.info(f"Excel report generated: {output_file}")

def get_user_input():
    """
    Get user input for file selection and analysis options.
    """
    file_path = input("Enter the path to your CSV or Excel file: ")
    group_by = input("Enter the column name to group by: ")
    agg_column = input("Enter the column name to aggregate: ")
    output_file = input("Enter the name for the output Excel file: ")
    return file_path, group_by, agg_column, output_file

def main():
    """
    Main function to run the data analysis and reporting tool.
    """
    file_path, group_by, agg_column, output_file = get_user_input()
    
    # Read the file
    df = read_file(file_path)
    if df is None:
        return
    
    # Clean the data
    df_cleaned = clean_data(df)
    if df_cleaned is None:
        return
    
    # Analyze the data
    analysis = analyze_data(df_cleaned)
    if analysis is None:
        return
    
    # Group the data
    grouped_data = group_data(df_cleaned, group_by, agg_column)
    if grouped_data is None:
        return
    
    # Generate Excel report
    generate_excel_report(df_cleaned, analysis, grouped_data, output_file)
    
    # Create and save charts
    bar_chart = create_bar_chart(grouped_data, f"{agg_column} by {group_by}", group_by, agg_column)
    bar_chart.savefig("bar_chart.png")
    logger.info("Bar chart saved as 'bar_chart.png'")
    
    line_chart = create_line_chart(df_cleaned[agg_column].rolling(window=5).mean(), 
                                   f"Rolling Mean of {agg_column}", "Index", f"Rolling Mean {agg_column}")
    line_chart.savefig("line_chart.png")
    logger.info("Line chart saved as 'line_chart.png'")
    
    logger.info("Data analysis and reporting completed successfully.")

In [6]:
# Run the data analysis and reporting tool
if __name__ == "__main__":
    main()

Enter the path to your CSV or Excel file:  sample_sales_data.csv
Enter the column name to group by:  Product
Enter the column name to aggregate:  Sales
Enter the name for the output Excel file:  Sales.xlsx


2025-01-30 22:42:42,727 - INFO - Excel report generated: Sales.xlsx
2025-01-30 22:42:44,824 - INFO - Bar chart saved as 'bar_chart.png'
2025-01-30 22:42:46,818 - INFO - Line chart saved as 'line_chart.png'
2025-01-30 22:42:46,823 - INFO - Data analysis and reporting completed successfully.


## How to Use the Data Analysis and Reporting Tool

1. Run the cell containing the main function.
2. When prompted, enter the path to your CSV or Excel file.
3. Specify the column you want to group by.
4. Specify the column you want to aggregate.
5. Enter a name for the output Excel file.

The tool will then:
- Read and clean the data
- Perform basic analysis
- Group the data as specified
- Generate an Excel report with multiple sheets
- Create and save bar and line charts

You can find the generated Excel report and chart images in the same directory as this notebook.